In [ ]:
!pip install --quiet arcgis

In [4]:
import os
from arcgis.gis import GIS;
import pandas as pd;
import requests;

In [18]:
# Pull the list of organization identifiers
org_url = r"https://attains.epa.gov/attains-public/api/domains";
response = requests.get(
     org_url
    ,params={
         'domainName': 'OrgStateCode'
    }
);
resp_json = response.json();

orgs = [];
for item in resp_json:
    
    if item['context'] == "EPA" or item['context'][:5] == "TEST_":
        continue;

    orgs.append(item);
    
df = pd.DataFrame(orgs)                                     \
    .drop(columns=['domain','name'])                        \
    .rename(columns={'code':'state','context':'orgid'})     \
    .sort_values(by=['state','orgid']);
    
#df

,state,orgid
54,AK,AKDECWQ
30,AL,21AWIC
61,AR,ARDEQH2O
34,AS,21AS
66,AZ,21ARIZ
32,AZ,NNEPA
37,CA,CA_BVR
64,CA,CA_SWRCB
31,CA,HVTEPA
40,CO,21COL001


In [ ]:
# For each orgid pull the assessments into a table
ste_url = r"https://attains.epa.gov/attains-public/api/assessmentUnits"

for index, row in df.iterrows():
    
    response = requests.get(
         ste_url
        ,params={
             'organizationId': row.orgid
        }
    );
    resp_json = response.json();
    
    cycle_year = resp_json['items'][0]['reportingCycleText'];
    
    assmnt = [];
    uses   = [];
    parms  = [];
    srcs   = [];
    docs   = [];
    
    for item in resp_json['items'][0]['assessments']:
        rec.append({
             "organizationIdentifier": row.orgid
            ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
            ,'reportingCycle': cycle_year
            ,"agencyCode": item['agencyCode']
            ,"trophicStatusCode": item['trophicStatusCode']
            ,"epaIRCategory": item['epaIRCategory']
        });
        
        if item['useAttainments'] is not None and len(item['useAttainments']) > 0:
            for uitme in item['useAttainments']:
                uses.append(
                     "organizationIdentifier": row.orgid
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"useName": uitem['useName']
                    ,"useAttainmentCode": uitem['useAttainmentCode']
                    ,"threatenedIndicator": uitem['threatenedIndicator']
                    ,"trendCode": uitem['trendCode']
                    ,"agencyCode": uitem['agencyCode']
                    ,"assessmentMetadata": uitem['assessmentMetadata']
                );
                
        if item['parameters'] is not None and len(item['parameters']) > 0:
            for pitme in item['parameters']:
                parms.append(
                     "organizationIdentifier": row.orgid
                    ,"assessmentUnitIdentifier": item['assessmentUnitIdentifier']
                    ,"reportingCycle": cycle_year
                    ,"parameterStatusName": pitem['parameterStatusName']
                    ,"parameterName": pitem['parameterName']
                    ,"pollutantIndicator": pitem['pollutantIndicator']
                );
                
                if pitem['associatedUses'] is not None and len(pitem['associatedUses']) > 0:
                    for auitem in pitem['associatedUses']:
        
    